<h1 align="center" style="color:green;font-size: 3em;">
Implementing Quantization Techniques</h1>

In this notebook, we will explore quantization techniques to optimize memory requirements.

### Install dependencies

In [2]:
%pip install datasets -q

Note: you may need to restart the kernel to use updated packages.


### Import Libraries

In [1]:
import torch

from transformers import BertModel, BertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
from torch.optim import AdamW
import torch.quantization

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'